In [ ]:
import os
import pickle
import keras as K
import numpy as np
import tensorflow as tf
import jams
import builtins
from tqdm import tqdm
import pandas as pd
import sys

builtins.tf = tf

In [ ]:
project_root = os.path.abspath(os.path.join(os.getcwd(), "../code"))

# Add to Python path if not already there
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import round_observation_times

In [ ]:
working = "ADDRESS_TO_YOUR_WORKING_DIRECTORY"

In [ ]:
def load_model(model_spec_path, weights_path):
    # Load the serialized model spec
    with open(model_spec_path, "rb") as fd:
        model_spec = pickle.load(fd)
    # Reconstruct the model from the serialized spec
    model = K.utils.deserialize_keras_object(model_spec, safe_mode=False)
    # Load the weights from the saved file
    model.load_weights(weights_path)
    return model

In [ ]:
output_path = os.path.join(working, "model_deep")
split = 0
epochs = 100

model_spec_path = os.path.join(
    output_path, "fold{:02d}_model_{:03d}_epochs.pkl".format(split, epochs)
)
weights_path = os.path.join(
    output_path, "fold{:02d}_weights_{:03d}_epochs.keras".format(split, epochs)
)

model = load_model(model_spec_path, weights_path)
model.summary()

In [ ]:
with open(
    os.path.join(working + "/pump.pkl"),
    "rb",
) as fd:
    pump = pickle.load(fd)

In [ ]:
def score_model(pump, model, idx, working, refs):
    results = {}
    for item in tqdm(idx, desc="Evaluating the model"):
        jam = jams.load(os.path.join(refs, f"{item}.jams"), validate=False)
        datum = np.load(os.path.join(working, "pump", f"{item}.npz"))[
            "cqt/mag"
        ]

        output = model.predict(datum)[0]

        ann = pump["chord_tag"].inverse(output)
        ann = round_observation_times(ann)

        ref_ann = round_observation_times(
            jam.annotations["chord", 0], precision=10
        )

        try:
            results[item] = jams.eval.chord(ref_ann, ann)
        except Exception as e:
            print(f"Error evaluating {item}: {e}")

    return pd.DataFrame.from_dict(results, orient="index")[
        ["root", "thirds", "triads", "tetrads", "mirex", "majmin", "sevenths"]
    ]

In [ ]:
test_dataset_path = "ADDRESS_TO_YOUR_CSV_TEST_DATASET"
idx = pd.read_csv(
    test_dataset_path,
    header=None,
    names=["id"],
)
pump_path = "ADDRESS_TO_YOUR_PUMP_DIRECTORY"
refs = "ADDRESS_TO_YOUR_ANNOTATIONS_REFERENCE_DIRECTORY"
scores = score_model(pump, model, idx["id"], pump_path, refs)

In [ ]:
scores.mean()

In [ ]:
test_audio_file_path = "ADDRESS_TO_YOUR_TEST_AUDIO_FILE"
input = pump.transform(test_audio_file_path)["cqt/mag"]

predictions = model.predict(input)[0]

ann = pump["chord_tag"].inverse(predictions)
print(ann)